In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#code to webscrape PC for race class

url = "https://www.procyclingstats.com/races.php?year=2025&month=&xnation=&filter=Filter&p=gravel&s=gravel-calendar"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", {"class": "basic"})

headers = [th.text.strip() for th in table.find_all("th")]

data = []
for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    data.append([cell.text.strip() for cell in cells])

df = pd.DataFrame(data, columns=headers)
df.to_excel("uci_gravel_race_classifications_2025.xlsx", index=False)

In [ ]:
core_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\structured_course_data.xlsx")
core_df

In [ ]:
class_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\Output\uci_race_classifications_total.xlsx")
class_df = class_df.rename(columns={'Race': 'Race Name', 'Class': 'Race Level'})
class_df

In [ ]:
from thefuzz import process

In [ ]:
def match_race_name(race_name, choices, threshold=90):
    match, score = process.extractOne(race_name, choices)
    return match if score >= threshold else None

In [ ]:
core_df['Matched Race Name'] = core_df['Race Name'].apply(lambda x: match_race_name(x, class_df['Race Name'].tolist()))

In [ ]:
merged_df = core_df.merge(class_df, left_on='Matched Race Name', right_on='Race Name', how='left', suffixes=('_old', ''))

In [ ]:
merged_df = merged_df.drop(columns=['Race Name_old', 'Matched Race Name']).rename(columns={'Race Name': 'Race Name', 'Race Level': 'Race Level'})

In [ ]:
merged_df.to_excel('reset_threshold_matched_Race_Name.xlsx')

In [ ]:
core_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\core.xlsx")

In [ ]:
core_df

In [ ]:
core_df = core_df.drop('Race Level_old', axis=1)

In [ ]:
core_df = core_df.drop('Unnamed: 0', axis=1)	

In [ ]:
core_df = core_df[['Race Name', 'Dates', 'Race Type', 'Race Level', 'Gender', 'Distance', 'Street',	'Road',	'Paved','Asphalt', 'Path', 'Cycleway', 'Unpaved', 'State Road',	'Cobblestones',	'Unknown', 'Compacted Gravel', 'Off-grid (unknown)', 'Singletrack',	'Access Road', 'Alpine', 'Net Gain', 'Lowest Elevation', 'Highest Elevation', 'Vertical Gain', 'Downhill'
]]

In [ ]:
core_df.to_excel('core.xlsx')

In [ ]:
no_class_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\core_dataframe.xlsx")
no_class_df

In [ ]:
no_class_df['Year'] = no_class_df['Race Name'].str.extract(r'(\d{4})')

In [ ]:
no_class_df['Year'].unique()

In [ ]:
no_class_df['Race Name'] = no_class_df['Race Name'].str.replace(r'\d{4}', '', regex=True).str.strip()

In [ ]:
no_class_df['Race Level'] = ''

In [ ]:
import json
with open(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\race_dict.json", "r") as f:
    race_dict = json.load(f)

In [ ]:
no_class_df['Race Level'] = no_class_df['Race Name'].map(race_dict)

In [ ]:
no_class_df['Race Level'].unique()

In [ ]:
no_class_df['Race Level'].describe()

In [ ]:
no_class_df['Matched Race Name'] = no_class_df['Race Name'].apply(lambda x: match_race_name(x, list(race_dict.keys())))

In [ ]:
no_class_df['Race Level'] = no_class_df['Matched Race Name'].map(race_dict)

In [ ]:
no_class_df['Race Name'] = no_class_df['Matched Race Name'].fillna(no_class_df['Race Name'])

In [ ]:
no_class_df['Race Level'].unique()

In [ ]:
no_class_df['Matched Race Name'] = no_class_df['Race Name'].apply(lambda x: match_race_name(x, class_df['Race Name'].tolist()))
no_class_df = no_class_df.merge(class_df, left_on='Matched Race Name', right_on='Race Name', how='left', suffixes=('_old', ''))
no_class_df['Race Name'] = no_class_df['Matched Race Name'].fillna(no_class_df['Race Name'])
no_class_df['Race Level'] = no_class_df['Race Level'].fillna(no_class_df['Race Level_old'])

In [ ]:
no_class_df = no_class_df.drop(columns=['Race Level_old', 'Matched Race Name'])

In [ ]:
no_class_df = no_class_df.rename(columns={'Race Name_old':'Race Name'})

In [ ]:
no_class_df.to_excel(r'no_class_df.xlsx')

In [ ]:
gravel = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\uci_gravel_race_classifications.xlsx")

In [ ]:
gravel = gravel[gravel['Category'] == 'ME']

In [ ]:
gravel.to_excel('mens_gravel_races.xlsx')